In [1]:
import torch.nn as nn
import torch
import torch.nn.utils.rnn as rnn
import statistics
import nltk # 없으시면 설치하세요: pip install nltk
import random
import collections
import time

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
## Dictionary class 선언
# 사전 만들어 주는 클래스 
class Dictionary(object):
    def __init__(self, dataset, size):
        ## init vocab ##
        self.word2idx = {'<pad>':0, '<sos>': 1, '<eos>': 2, '<unk>': 3} # 사전 
        # sos는 start eos는 end unk는 사전에 입력되지 않은 거 처리
        self.idx2word = ['<pad>', '<sos>', '<eos>', '<unk>'] # inverted dictionary
        # self.word2idx['<unk>'] --> 3 을 출력 키 값을 인덱스를 줌
        # self.idx2word[3] = '<unk>'을 출력 인덱스로 단어를 줌
        
        self.build_dict(dataset, size)
    
    def __call__(self, word):
        return self.word2idx.get(word, self.word2idx['<unk>']) # if word does not exist in vocab then return unk idx

    def add_word(self, word):
        if word not in self.word2idx:
            self.idx2word.append(word)
            self.word2idx[word] = len(self.idx2word) - 1
        return self.word2idx[word]

    def build_dict(self, dataset, dict_size):
        ## Practice ##
        """Tokenize a text file."""
        # (the, the, a, boy, girl, ..,)
        total_words = (word for sent in dataset for word in sent) # store all words into tuple 
        # 데이터셋에 있는 것을 튜플로 모두 담는 것 = total_words
        
        # ('The': 10000, 'a': 5555, 'girl': 3400, ...)
        word_freq = collections.Counter(total_words)# count the number of each word: ex) 
        # 빈도수를 셈 = word_freq
        
        vocab = sorted(word_freq.keys(), key=lambda word: (word_freq[word], word), reverse = True ) # Practice: sort by frequency
        vocab = vocab[:dict_size] # truncate  사전에 넣을 것을 짜르기
        for word in vocab:
            self.add_word(word)

    def __len__(self):
        return len(self.idx2word)


In [4]:
## Brown dataset Preprocessing (NLTK)
def brown_dataset(min=5, max=30):
    nltk.download('brown')

    # get sentences with the length between min and max
    # convert all words into lower-case
    all_seq = [[token.lower() for token in seq] for seq in nltk.corpus.brown.sents() 
               if min <= len(seq) <= max]

    random.shuffle(all_seq) # shuffle
    return all_seq

In [5]:
## Download Brown dataset
dataset = brown_dataset()
print(len(dataset))
## print some part
print(dataset[0])
print(dataset[1])
print(dataset[2])

[nltk_data] Downloading package brown to /home/piai/nltk_data...
[nltk_data]   Package brown is already up-to-date!


43450
['keane', 'left', ',', 'within', 'three', 'days', '.']
['cattle', 'drove', 'to', 'the', 'northern', 'ranges', 'and', 'held', 'for', 'two', 'winters', 'to', 'mature', "'em", 'into', 'prime', 'beef', 'were', 'said', 'to', 'be', '``', 'double', 'wintered', "''", '.']
['such', 'a', 'twist', 'would', 'be', 'a', 'tragedy', 'for', 'the', 'dominican', 'people', ',', 'who', 'deserve', 'to', 'breathe', 'without', 'fear', '.']


In [6]:
## Data handler class 선언
class Corpus(object):
    def __init__(self, dataset, device, dict_size=20000, train_ratio=0.97):
        train_size = int(len(dataset) * train_ratio)
        self.device = device
        self.dictionary = Dictionary(dataset, dict_size)
        self.train = dataset[:train_size] # [0 ~ train_size]
        self.valid = dataset[train_size:] # [train_size: len(dataset)]

    def indexing(self, dat):  # batching 인덱싱 하는 함수
        # dat = list(list)   [['we', 'are', 'the', 'word'], ["hi", "my", "name", ...""]]
        src_idxes = [] # 모델 입력
        tgt_idxes = [] # 모델 정답
        for sent in dat:
            src_idx = [self.dictionary('<sos>')]+ [self.dictionary(word) for word in sent]
            tgt_idx = [self.dictionary(word) for word in sent] + [self.dictionary('<eos>')]
            src_idxes.append(torch.tensor(src_idx).type(torch.int64))
            tgt_idxes.append(torch.tensor(tgt_idx).type(torch.int64))
        
        # shape = [B, L]   pading 넣어주기
        src_idxes = rnn.pad_sequence(src_idxes, batch_first=True).to(self.device) # [batch, length]
        
        # flatten shape = [B * L]
        tgt_idxes = rnn.pad_sequence(tgt_idxes, batch_first=True).to(self.device).view(-1) # [batch * length]

        return src_idxes, tgt_idxes

    def batch_iter(self, batch_size, isTrain=True):
        dat = self.train if isTrain else self.valid
        if isTrain:
            random.shuffle(dat)

        for i in range(len(dat) // batch_size):
            batch = dat[i * batch_size: (i+1) * batch_size]
            src, tgt = self.indexing(batch)
            yield {'src': src, 'tgt': tgt}

In [7]:
corpus = Corpus(dataset, device)

In [8]:
# Dictionary 확인
for i, (key, val) in enumerate(corpus.dictionary.word2idx.items()):
    print('word:  {:10s} | index: {:5d} '.format(key, val))
    if i == 20:
        break

word:  <pad>      | index:     0 
word:  <sos>      | index:     1 
word:  <eos>      | index:     2 
word:  <unk>      | index:     3 
word:  the        | index:     4 
word:  .          | index:     5 
word:  ,          | index:     6 
word:  of         | index:     7 
word:  and        | index:     8 
word:  to         | index:     9 
word:  a          | index:    10 
word:  in         | index:    11 
word:  was        | index:    12 
word:  he         | index:    13 
word:  is         | index:    14 
word:  ''         | index:    15 
word:  ``         | index:    16 
word:  it         | index:    17 
word:  that       | index:    18 
word:  for        | index:    19 
word:  ;          | index:    20 


In [9]:
## indexing 함수 결과 확인

# case : 단일 문장 입력 시. 
sent = [dataset[1]]
idx_src, idx_tgt = corpus.indexing(sent)

print(sent)
print(idx_src) # <SOS> index로 시작
print(idx_tgt) # <EOS> index로 종료

print('-' * 90)
## case : 복수 문장 입력 시 (batching)
batch = [dataset[0], dataset[1]]
idx_src, idx_tgt = corpus.indexing(batch)

print(batch)
print(idx_src) # 가장 길이가 긴 문장 (dataset[0]) 보다 짧은 문장 (dataset[1]) 의 경우 남는 길이만큼 padding=0 삽입 확인.
print(idx_tgt)

[['after', 'his', 'pains', 'got', 'worse', ',', 'tom', 'decided', 'to', 'see', 'a', 'real', 'doctor', ',', 'from', 'whom', 'he', 'learned', 'he', 'was', 'suffering', 'from', 'cancer', 'of', 'the', 'lung', '.']]
tensor([[   1,  108,   21, 5676,  162, 1809,    6, 1423,  703,    9,  124,   10,
          374,  904,    6,   36,  946,   13, 1050,   13,   12, 3703,   36, 4282,
            7,    4, 4978,    5]], device='cuda:0')
tensor([ 108,   21, 5676,  162, 1809,    6, 1423,  703,    9,  124,   10,  374,
         904,    6,   36,  946,   13, 1050,   13,   12, 3703,   36, 4282,    7,
           4, 4978,    5,    2], device='cuda:0')
------------------------------------------------------------------------------------------
[['--', 'in', 'a', 'physiochemical', 'study', 'of', 'seed', 'proteins', ',', 'the', 'globulins', 'of', 'the', 'brazil', 'nut', 'have', 'been', 'investigated', '.'], ['after', 'his', 'pains', 'got', 'worse', ',', 'tom', 'decided', 'to', 'see', 'a', 'real', 'doctor', ',', 'fr

In [19]:
# Practice

# Convert word indices to word with corpus.dictionary
word_indices = [13, 29, 56, 458, 120, 109, 78, 9149, 49, 239, 47, 5]
words = [corpus.dictionary.idx2word[idx] for idx in word_indices]
print(words)

['he', 'had', 'no', 'love', 'how', 'much', 'time', 'strasbourg', 'would', 'felt', 'him', '.']


# RNN Model
![모델구조](fig/LM_model.png)

In [9]:
## RNN Language model 선언

# Define network
class RNNModel(nn.Module):
    def __init__(self, ntoken, hidden_size, nlayers, dropout=0.1):
        super(RNNModel, self).__init__()
        # ntoken: size of vocab
        # hidden: size of hidden vector
        
        self.drop = nn.Dropout(dropout)
        self.embeddings = nn.Embedding(ntoken, hidden_size, padding_idx=0) # Practice 
        self.rnn = nn.LSTM(hidden_size, hidden_size, nlayers, 
                           dropout=dropout, batch_first=True) # Practice 
        self.output_layer = nn.Linear(hidden_size, ntoken) # Practice
        self.sm = nn.LogSoftmax(dim=-1) # log확률값

        self.ntoken = ntoken
        self.hidden_size = hidden_size
        self.nlayers = nlayers

        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        self.embeddings.weight.data.uniform_(-initrange, initrange)
        self.output_layer.weight.data.uniform_(-initrange, initrange)
        self.output_layer.bias.data.zero_()

    def forward(self, input, hidden):
        # shape(input) = [Batch, length]
        emb = self.embeddings(input) # input -> embedding = (batch, length, dim)
        output, hidden = self.rnn(emb, hidden) # output = (batch. length. dim)
        output = self.drop(output) #dropout
        output = self.output_layer(output)
        # linear projection : hidden dim --> vocab size (batch, len, vocab)
        output = output.view(-1, self.ntoken) # output = (batch * length, vocab_size)
        output = self.sm(output)
        return output, hidden

    def init_hidden(self, bsz): # 첫번째 hidden의 history (h(-1))
        weight = next(self.parameters()) # to set init tensor with the same torch.dtype and torch.device
        return (weight.new_zeros(self.nlayers, bsz, self.hidden_size),
                weight.new_zeros(self.nlayers, bsz, self.hidden_size))


In [10]:
# Hyperparameters
batch_size = 30 # 자기가 설정 가능
hidden_size = 256
dropout = 0.2
max_epoch = 30

# build model
ntokens = len(corpus.dictionary)
model = RNNModel(ntokens, hidden_size, 1, dropout).to(device)

#isTrain=True # Flag variable
isTrain=False # 이미 학습한 모델을 단순히 load만 수행하고 학습 X

# set loss func and optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
criterion = nn.NLLLoss(ignore_index=0, reduction='mean')

/home/piai/anaconda3/envs/AIBD/lib/python3.8/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [11]:
##### Training / Evaluation Parts #######

In [12]:
# accuracy
def cal_acc(scores, target):
    pred = scores.max(-1)[1]
    non_pad = target.ne(0)
    num_correct = pred.eq(target).masked_select(non_pad).sum().item() 
    num_non_pad = non_pad.sum().item()
    return 100 * (num_correct / num_non_pad)

# Training
![학습](fig/LM_train.png)

In [13]:
# train func.
def train():
    model.train() # Turn on training mode which enables dropout.
    mean_loss = []
    mean_acc = []
    start_time = time.time()

    for batch in corpus.batch_iter(batch_size):
        hidden = model.init_hidden(batch_size) # zero vectors for init hidden
        target = batch['tgt'] # flattened target 
        optimizer.zero_grad()
        output, hidden = model(batch['src'], hidden) # output = flatten output = [Batch_size * Length, vocab_size]

        # output shape = (batch * length, vocab_size)
        # target shape = (batch * length)   --> (batch * length, vocab_size) 로 one-hot distribtuion으로 내부적으로 변환되어 비교 수행
        loss = criterion(output, target) # compare between vocab_prob and answer_prob(one-hot converted)
        loss.backward()
        optimizer.step()

        mean_loss.append(loss.item())
        mean_acc.append(cal_acc(output, target))

    total_time = time.time() - start_time
    mean_acc = statistics.mean(mean_acc)
    mean_loss = statistics.mean(mean_loss)

    return mean_loss, total_time, mean_acc

In [14]:
# evaluation func.
def evaluate():
    model.eval() # Turn off dropout
    mean_loss = []
    mean_acc = []

    for batch in corpus.batch_iter(batch_size, isTrain=False):
        with torch.no_grad():
            hidden = model.init_hidden(batch_size)
            target = batch['tgt']
            output, hidden = model(batch['src'], hidden)
            loss = criterion(output, target)
            mean_loss.append(loss.item())
            mean_acc.append(cal_acc(output, target))

    mean_acc = statistics.mean(mean_acc)
    mean_loss = statistics.mean(mean_loss)

    return mean_loss, mean_acc

In [20]:
if isTrain: # set False if you don't need to train model
    start_time = time.time()

    for epoch in range(1, max_epoch+1):
        loss, epoch_time, accuracy = train()
        print('epoch {:4d} | times {:3.3f} |  loss: {:3.3f} | accuracy: {:3.2f}'.format(epoch+1, epoch_time, loss, accuracy))

        if epoch % 10 == 0:
            loss, accuracy = evaluate()
            print('=' * 60)
            print('Evaluation | loss: {:3.3f} | accuracy: {:3.2f}'.format(loss, accuracy))
            print('=' * 60)

    with open('model.pt', 'wb') as f:
        print('save model at: ./model.pt')
        torch.save(model, f)

epoch    2 | times 16.682 |  loss: 5.618 | accuracy: 20.95
epoch    3 | times 16.734 |  loss: 5.034 | accuracy: 24.08
epoch    4 | times 16.734 |  loss: 4.689 | accuracy: 25.63
epoch    5 | times 16.784 |  loss: 4.389 | accuracy: 27.04
epoch    6 | times 16.806 |  loss: 4.130 | accuracy: 28.54
epoch    7 | times 16.861 |  loss: 3.914 | accuracy: 30.03
epoch    8 | times 16.890 |  loss: 3.741 | accuracy: 31.52
epoch    9 | times 16.980 |  loss: 3.603 | accuracy: 32.77
epoch   10 | times 16.880 |  loss: 3.494 | accuracy: 33.87
epoch   11 | times 16.457 |  loss: 3.408 | accuracy: 34.70
Evaluation | loss: 5.987 | accuracy: 23.74
epoch   12 | times 16.316 |  loss: 3.338 | accuracy: 35.46
epoch   13 | times 16.405 |  loss: 3.280 | accuracy: 36.13
epoch   14 | times 17.026 |  loss: 3.234 | accuracy: 36.67
epoch   15 | times 17.055 |  loss: 3.198 | accuracy: 37.03
epoch   16 | times 17.037 |  loss: 3.165 | accuracy: 37.41
epoch   17 | times 17.042 |  loss: 3.140 | accuracy: 37.67
epoch   18 | 

# 과제1: 문장 확률 계산
![문장확률](fig/sent_prob_1.png)
![문장확률](fig/sent_prob_2.png)

In [20]:
def pred_sent_prob(sent):
    model.eval()
    with torch.no_grad():
        # 1. 모델 입력 및 정답 문장에 대한 단어 indexing
        idx_src, idx_tgt = corpus.indexing(sent)
        
        # 2. initial hidden 생성
        hidden = model.init_hidden(1)
        
        # 3. LM의 결과(확률분포) 생성
        output, hidden = model(idx_src, hidden)
        
        # 4. 모델 확률분포로부터 정답 단어의 각 index에 대한 Log 확률 값 추출.        
        prob=0
        target_list=idx_tgt.tolist()
        
        # 5. log 확률의 합.
        for i in range(len(target_list)):
            prob+=output[i][target_list[i]]
            
        # 6. 결과 return (return type: float)
        return prob

In [21]:
# load saved model
with open('./model.pt', 'rb') as f:
    print('load model from: ./model.pt')
    model = torch.load(f).to(device)

    print('log prob of [the dog bark .]: {:3.3f}'.format(pred_sent_prob([['the', 'dog', 'bark', '.']])))
    print('log prob of [the cat bark .]: {:3.3f}'.format(pred_sent_prob([['the', 'cat', 'bark', '.']])))

    print('log prob of [boy am a i .]: {:3.3f}'.format(pred_sent_prob([['boy', 'am', 'a', 'i', '.']])))
    print('log prob of [i am a boy .]: {:3.3f}'.format(pred_sent_prob([['i', 'am', 'a', 'boy', '.']])))


load model from: ./model.pt
log prob of [the dog bark .]: -51.045
log prob of [the cat bark .]: -55.037
log prob of [boy am a i .]: -34.454
log prob of [i am a boy .]: -19.546


# 과제2: 다음 단어 예측
![다음단어예측](fig/next_word.png)

In [22]:
def pred_next_word(partial_sent, topN=3):
    model.eval()
    topN_word = []
    with torch.no_grad():
        # 1. 모델 입력 및 정답 문장에 대한 단어 indexing
        idx_src, idx_tgt = corpus.indexing(partial_sent)
        
        # 2. initial hidden 생성
        hidden = model.init_hidden(1)
        
        # 3. LM의 결과(확률분포) 생성
        output, hidden = model(idx_src,hidden)

        
        # 4. topN에 해당하는 다음단어의 word index 추출 (Hint: torch.topk() 활용)
        val,idx=torch.topk(output[-1],topN)

        # 5. word index --> word 로 변환
        for (key, val) in (corpus.dictionary.word2idx.items()):
            for j in range(len(idx)):
                if idx[j] == val:
                    topN_word.append(key)
        
        # 6. topN word list 반환 (return type: list)
            
        return topN_word

In [23]:
partial_sent = [['the', 'next', 'word']]
N=3
candidates = pred_next_word(partial_sent, topN=N)

# print 
partial_sent = ' '.join(partial_sent[0])
print('Top {0} next words for a partial sentence [{1}] is: '.format(N, partial_sent))
print('===>', candidates)

Top 3 next words for a partial sentence [the next word] is: 
===> [',', 'of', 'is']
